# readME

MIMIC_IV sepsis3 defined sepsis shock



从postgresql数据库中查询(**数据在fjmu server中**) 定义脓毒症休克 需要的指标 并存在 pickle_mimic_iv_derived_table.pkl 中 


Vasopressors 修改为  

 221289  --Epinephrine
,221653  --Dobutamine
,221662  --Dopamine
,221749   --Phenylephrine
,221906   --Norepinephrine
,222315   --Vasopressin
,227692   --Isuprel
,229617   --Epinephrine.
,229630   --Phenylephrine (50/250)
,229631   --Phenylephrine (200/250)_OLD_1
,229632   --Phenylephrine (200/250)
,229764   --Angiotensin II (Giapreza)
,229709   --Angiotensin II (Giapreza)

并存于 **my_medication_vasopressor_v3** 中

**my_fluid_colloid_and_crystalloid_all2** 比1 多了   mv.rate is not null

包括:
- mimiciv_icu: <br>
icustays

- mimiciv_derived: <br>
sepsis3、icustay_detail

- user_刘光豪: <br>
my_age、my_fluid_colloid_and_crystalloid_all2、my_mbp2、my_icu_intersect_lactate2、my_medication_vasopressor_v3




In [4]:
import pandas as pd
import os
import psycopg2
import time

import nbimporter
#import my_save_load
os.chdir(r'D:\my_dir_jupyter\final_predict_septic_shock')
import my_save_load_v2


In [5]:
#提取fjmu 中的数据
def get_pg_info(sql):
    conn = psycopg2.connect(host="localhost", user="postgres", password="pgs.123456.", port=5555, database="mimiciv_20")
    cur = conn.cursor()
    cur.execute(sql)
    colnames = [desc[0] for desc in cur.description]
    data = pd.DataFrame(cur.fetchall(), columns=colnames)
    conn.close()
    return data


def get_table(mode, table_name):
    sql = 'SELECT * FROM ' + mode + '.' + table_name
    table = get_pg_info(sql)
    return (table)


# 导入所需表格

In [9]:

###### define  septic shock by using tables below
icustays = get_table(mode='mimiciv_icu', table_name='icustays')
sepsis3_original = get_table(mode='mimiciv_derived', table_name='sepsis3')
icustays_detail = get_table(mode='mimiciv_derived', table_name='icustay_detail')


age = get_table(mode='user_刘光豪', table_name='my_age')
fluids_bolus = get_table(mode='user_刘光豪', table_name='my_fluid_colloid_and_crystalloid_all2')  # 综合胶体和晶体复苏
fluids_bolus = fluids_bolus.loc[:,['stay_id', 'starttime', 'endtime', 'colloid_bolus','patientweight']]

shock_define_mbp = get_table(mode='user_刘光豪', table_name='my_mbp2')
shock_define_lactate2 = get_table(mode='user_刘光豪', table_name='my_icu_intersect_lactate2')
vasoactive_vasopressor = get_table(mode='user_刘光豪', table_name='my_medication_vasopressor_v3')  # v3


# 去除ICU 住院小于24H的患者和年龄小于18岁的患者 

In [ ]:
# 去除ICU 住院小于24H的患者和年龄小于18岁的患者
# sepsis3_original = pd.merge(icustays_detail.loc[(icustays_detail.los_icu >= 1) & (icustays_detail.admission_age >=18 ),'stay_id'],sepsis3_original,on='stay_id')


# 保存数据

In [10]:
###### save file
path=r'E:\data_phenotype_for_septic_shock'  #


os.chdir(path)

# out_list = [ 'age', 'fluids_bolus', 'fluids_colloid_bolus', 'fluids_crystalloid_bolus', 'icustays', 'sepsis3_original','shock_define_lactate2', 'shock_define_mbp',  'vasoactive_vasopressor']
out_list = [ 'age', 'fluids_bolus', 'icustays', 'sepsis3_original','shock_define_lactate2', 'shock_define_mbp',
            'vasoactive_vasopressor']

out_filename = r'pickle_mimic_iv_derived_table_v5.pkl'

my_save_load_v2.my_save(out_list, out_filename, globals())


